# Setup

msticpy is a set of Python tools intended to be used for security investigations and hunting. Many of the tools originated as code Jupyter notebooks written to solve a problem as part of a security investigation. Some of the tools are only useful in notebooks (e.g., the widgets and visualizations) but many others can be used from the Python command line or imported into your code.

MSTICpy includes a feature called `nbinit` that handles the process of installing and importing modules into a notebook environment. This was developed to allow for a clearer starting cell in notebooks and to avoid users being presented with a very large cell block at the top of a notebook. By passing the notebook namespace to `init_notebook()`, this function handles the job of installing and importing core MSTICpy packages along with any others that might be needed by a notebook.

Find more details for the installation from [MSTICPY](https://msticpy.readthedocs.io/en/latest/getting_started/Installing.html#).

```markdown
The code snippet below initializes the MSTICpy environment and imports additional packages and modules required for the notebook. Here's a detailed explanation:

```python
from msticpy.nbtools import nbinit
```
- This line imports the `nbinit` module from `msticpy.nbtools`. The `nbinit` module is used to initialize the notebook environment with necessary imports and configurations.

```python
extra_imports = [
    "msticpy.vis.timeseries, display_timeseries_anomolies",
    "msticpy.analysis.timeseries, timeseries_anomalies_stl",
    "datetime, datetime",
    "msticpy.vis.nbdisplay, draw_alert_entity_graph",
    "msticpy.context.ip_utils, convert_to_ip_entities",
    "msticpy.vis.ti_browser, browse_results",
    "IPython.display, Image",
    "msticpy.context.ip_utils, get_whois_info",
    "msticpy.context.ip_utils, get_ip_type"
]
```
- This list, `extra_imports`, contains strings that specify additional modules and functions to be imported. Each string is in the format `"module, function"`, indicating the module and the specific function to import from that module.

```python
nbinit.init_notebook(
    namespace=globals(),
    additional_packages=["pyvis"],
    extra_imports=extra_imports,
);
```
- The `init_notebook` function from `nbinit` is called to initialize the notebook environment.
  - `namespace=globals()`: This argument passes the global namespace to the function, allowing it to import modules and functions into the global scope of the notebook.
  - `additional_packages=["pyvis"]`: This argument specifies additional packages to be installed and imported, in this case, the `pyvis` package.
  - `extra_imports=extra_imports`: This argument passes the list of additional imports defined earlier.

```python
from msticpy.context import TILookup
ti = TILookup()
```
- This line imports the `TILookup` class from `msticpy.context` and creates an instance of it named `ti`. The `TILookup` class is used for threat intelligence lookups, allowing the notebook to query threat intelligence data.
```
```

In [ ]:
from msticpy.nbtools import nbinit
extra_imports = [
    "msticpy.vis.timeseries, display_timeseries_anomolies",
    "msticpy.analysis.timeseries, timeseries_anomalies_stl",
    "datetime, datetime",
    "msticpy.vis.nbdisplay, draw_alert_entity_graph",
    "msticpy.context.ip_utils, convert_to_ip_entities",
    "msticpy.vis.ti_browser, browse_results",
    "IPython.display, Image",
    "msticpy.context.ip_utils, get_whois_info",
    "msticpy.context.ip_utils, get_ip_type"
    
]

nbinit.init_notebook(
    namespace=globals(),
    additional_packages=["pyvis"],
    extra_imports=extra_imports,
);

from msticpy.context import TILookup
ti = TILookup()

In [ ]:
# Initalize and connect to Azure Sentinel using details from our config file.
qry_prov = QueryProvider('LogAnalytics')
wkspace = WorkspaceConfig()
qry_prov.connect(wkspace.code_connect_str)

# Threat Intelligence enrichments 
Finding a Threat Intelligence Indicators for tracking the IP address involements in any cloud operations, Here you can utilise a your owen threat intelligence feeds or API calls, we are using a Tredinals TI based feeds from sentinel loag analytics tables


The following code performs a query to retrieve threat intelligence indicators from the `ThreatIntelligenceIndicator` table. The query filters for indicators generated in the last 30 days with a confidence score of 80 or higher. It extends the data with additional fields to classify the type of indicator (Domain or NetworkIP) and consolidates various indicator fields into a single field (`TIIoc`). The query excludes private IP addresses and ensures that the indicator field is not empty. Finally, it selects distinct records based on threat type, indicator type, indicator value, and activity group names. The results are executed and converted into a dataframe for display.

```python
query = f"""ThreatIntelligenceIndicator
|where TimeGenerated >= ago(30d)
|where ConfidenceScore >= 80
|extend NetworkIOCType= iff(isnotempty(DomainName),"Domain","NetworkIP")
|extend IndicatorType= coalesce(FileHashType,NetworkIOCType), TIIoc= coalesce(NetworkSourceIP,FileHashValue,DomainName)
|where not(isempty(TIIoc)) and not(ipv4_is_private(TIIoc))
|distinct ThreatType,IndicatorType,TIIoc,ActivityGroupNames
"""

ThreatintelligenceIOCs = qry_prov.exec_query(query)
ThreatintelligenceIOCs = ThreatintelligenceIOCs.to_dataframe()
display(ThreatintelligenceIOCs)
```

Here we are focussing IP based tredinal Threat intelligence telemetry, hower we can expand more scope based on customised TI feeds as per Orgs using 

In [ ]:
query = f"""ThreatIntelligenceIndicator
|where TimeGenerated >= ago(30d)
|where ConfidenceScore >= 80
|extend NetworkIOCType= iff(isnotempty(DomainName),"Domain","NetworkIP")
|extend IndicatorType= coalesce(FileHashType,NetworkIOCType), TIIoc= coalesce(NetworkSourceIP,FileHashValue,DomainName)
|where not(isempty(TIIoc)) and not(ipv4_is_private(TIIoc))
|distinct ThreatType,IndicatorType,TIIoc,ActivityGroupNames
"""

ThreatintelligenceIOCs = qry_prov.exec_query(query)
ThreatintelligenceIOCs = ThreatintelligenceIOCs.to_dataframe()
display(ThreatintelligenceIOCs)

# Security Tables Normerlizations  

### AWS GuardDuty Findings and Normalization

The following Kusto Query Language (KQL) query is designed to normalize security data from AWS GuardDuty findings. This normalization process helps in standardizing the data, making it easier to analyze and correlate with other security data sources.

#### Explanation of the KQL Query:

1. **Time Filtering:**
    ```kql
    | where TimeCreated between ((referenceTime - lookBack) .. (referenceTime + lookAfter))
    ```
    - Filters the findings to include only those created within a specified time range.

2. **Excluding Test Files:**
    ```kql
    | where Title !has "EICAR-Test-File"
    ```
    - Excludes any findings related to the EICAR test file, which is commonly used for testing antivirus software.

3. **Activity Type Filtering:**
    ```kql
    | where array_length(activityTypes) > 0
    | where isnotempty(ActivityType)
    | where ActivityType in (activityTypes)
    ```
    - Ensures that the findings have valid activity types and filters based on specific activity types.

4. **Extracting and Transforming JSON Fields:**
    ```kql
    | extend
        ServiceDetails= tostring(parse_json(ServiceDetails)),
        ResourceDetails = tostring(parse_json(ResourceDetails))
    ```
    - Parses the `ServiceDetails` and `ResourceDetails` fields from JSON format to string format for further processing.

5. **Extracting IP Addresses:**
    ```kql
    | extend
        ApiCallIP= tostring(parse_json(ServiceDetails).awsApiCallAction.remoteIpDetails.ipAddressV4), 
        ApiCallIP2= tostring(parse_json(ServiceDetails).action.awsApiCallAction.remoteIpDetails.ipAddressV4), 
        NetworkConnectionIP = tostring(parse_json(ServiceDetails).action.awsApiCallAction.remoteIpDetails.ipAddressV4),
        InstanceRemoteIp= tostring(parse_json(ResourceDetails).instanceDetails.networkInterfaces[0].publicIp),
        InstancePORT_PROBERemoteIp= tostring(parse_json(ServiceDetails).action.portProbeAction.portProbeDetails[0].remoteIpDetails.ipAddressV4),
        InstanceNETWORK_CONNECTIONRemoteIp= tostring(parse_json(ServiceDetails).action.networkConnectionAction.remoteIpDetails.ipAddressV4),
        InstancePrivateIp= tostring(parse_json(ResourceDetails).instanceDetails.networkInterfaces[0].privateIpAddress),
        RdsLoginRemoteIp=tostring(parse_json(ServiceDetails).action.rdsLoginAttemptAction.remoteIpDetails.ipAddressV4),
        KubernetesRemoteIp = tostring(parse_json(ServiceDetails).action.kubernetesApiCallAction.remoteIpDetails.ipAddressV4),
        KubernetesPrivateIp = tostring(parse_json(ServiceDetails).action.kubernetesApiCallAction.sourceIPs[0])
    ```
    - Extracts various IP addresses from the parsed JSON fields.

6. **Extracting Additional Resource Details:**
    ```kql
    | extend
        ResourceType =  tostring(parse_json(ResourceDetails).resourceType),
        KubernetesUserId= tostring(parse_json(ResourceDetails).kubernetesDetails.kubernetesUserDetails.uid),
        RdsDbUserId= tostring(parse_json(ResourceDetails).rdsDbUserDetails.user),
        EksClusterId = tostring(parse_json(ResourceDetails).eksClusterDetails.vpcId),
        lambdaFunctionName = tostring(parse_json(ResourceDetails).lambdaDetails.functionName),
        eksClusterName = tostring(parse_json(ResourceDetails).eksClusterDetails.name),
        ResourceUserId = tostring(parse_json(ResourceDetails).accessKeyDetails.principalId),
        ResourceIamInstanceProfileId = tostring(parse_json(ResourceDetails).instanceDetails.iamInstanceProfile.id),
        ResourceInstanceId = tostring(parse_json(ResourceDetails).instanceDetails.iamInstanceProfile.id),
        ResourceName= tostring(parse_json(ResourceDetails).instanceDetails.instanceId),
        ResourceId= tostring(parse_json(ResourceDetails).instanceDetails.outpostArn),
        AwsActionType= tostring(parse_json(ServiceDetails).action.actionType)
    ```
    - Extracts additional details about the resources involved in the findings.

7. **Combining and Filtering IP Addresses:**
    ```kql
    | extend
        AWSRemoteNetworkIp= coalesce(ApiCallIP,ApiCallIP2, NetworkConnectionIP, InstanceRemoteIp, KubernetesRemoteIp, InstancePORT_PROBERemoteIp, InstanceNETWORK_CONNECTIONRemoteIp, RdsLoginRemoteIp), 
        AWSAssetNetworkIp=coalesce(InstancePrivateIp, KubernetesPrivateIp, "127.0.0.1"), 
        ResourceName = coalesce(ResourceName, ResourceId, eksClusterName, lambdaFunctionName, AwsActionType),
        ResourceUserId =  coalesce(ResourceUserId, ResourceIamInstanceProfileId, KubernetesUserId, EksClusterId, RdsDbUserId, "Na")
    | where not(isempty(AWSRemoteNetworkIp)) and not(isempty(AWSAssetNetworkIp)) and not(ipv4_is_private(AWSRemoteNetworkIp))
    ```
    - Combines various IP address fields into `AWSRemoteNetworkIp` and `AWSAssetNetworkIp`.
    - Filters out private IP addresses and ensures that the IP address fields are not empty.

8. **Selecting Distinct Records:**
    ```kql
    | distinct
        TimeGenerated,
        ActivityType,
        ResourceType,
        ResourceUserId,
        ResourceName,
        AWSRemoteNetworkIp,
        AWSAssetNetworkIp,
        Title,
        Description
    ```
    - Selects distinct records based on the specified fields to avoid duplicate entries.

### Execution and Display:
```python
AWSAlerts = qry_prov.exec_query(query)
AWSAlerts = AWSAlerts.to_dataframe()
display(AWSAlerts)
```
- Executes the query using the `qry_prov` object, converts the results into a dataframe, and displays the findings.

This query helps in normalizing AWS GuardDuty findings by extracting and transforming relevant fields, filtering out unnecessary data, and selecting distinct records. This standardized data can then be used for further analysis and correlation with other security data sources.


In [ ]:
# AWS Guardduity Findings and normerlization

query = f"""AWSGuardDuty
    | where TimeCreated between ((referenceTime - lookBack) .. (referenceTime + lookAfter))
    | where Title !has "EICAR-Test-File"
    | where array_length(activityTypes) > 0
    | where isnotempty(ActivityType)
    | where ActivityType in (activityTypes)
    | extend
        ServiceDetails= tostring(parse_json(ServiceDetails)),
        ResourceDetails = tostring(parse_json(ResourceDetails))
    | extend
        ApiCallIP= tostring(parse_json(ServiceDetails).awsApiCallAction.remoteIpDetails.ipAddressV4), ApiCallIP2= tostring(parse_json(ServiceDetails).action.awsApiCallAction.remoteIpDetails.ipAddressV4), 
        NetworkConnectionIP = tostring(parse_json(ServiceDetails).action.awsApiCallAction.remoteIpDetails.ipAddressV4),
        InstanceRemoteIp= tostring(parse_json(ResourceDetails).instanceDetails.networkInterfaces[0].publicIp),
        InstancePORT_PROBERemoteIp= tostring(parse_json(ServiceDetails).action.portProbeAction.portProbeDetails[0].remoteIpDetails.ipAddressV4),
        InstanceNETWORK_CONNECTIONRemoteIp= tostring(parse_json(ServiceDetails).action.networkConnectionAction.remoteIpDetails.ipAddressV4),
        InstancePrivateIp= tostring(parse_json(ResourceDetails).instanceDetails.networkInterfaces[0].privateIpAddress),
        RdsLoginRemoteIp=tostring(parse_json(ServiceDetails).action.rdsLoginAttemptAction.remoteIpDetails.ipAddressV4),
        KubernetesRemoteIp = tostring(parse_json(ServiceDetails).action.kubernetesApiCallAction.remoteIpDetails.ipAddressV4),
        KubernetesPrivateIp = tostring(parse_json(ServiceDetails).action.kubernetesApiCallAction.sourceIPs[0]),
        ResourceType =  tostring(parse_json(ResourceDetails).resourceType),
        KubernetesUserId= tostring(parse_json(ResourceDetails).kubernetesDetails.kubernetesUserDetails.uid),
        RdsDbUserId= tostring(parse_json(ResourceDetails).rdsDbUserDetails.user),
        EksClusterId = tostring(parse_json(ResourceDetails).eksClusterDetails.vpcId),
        lambdaFunctionName = tostring(parse_json(ResourceDetails).lambdaDetails.functionName),
        eksClusterName = tostring(parse_json(ResourceDetails).eksClusterDetails.name),
        ResourceUserId = tostring(parse_json(ResourceDetails).accessKeyDetails.principalId),
        ResourceIamInstanceProfileId = tostring(parse_json(ResourceDetails).instanceDetails.iamInstanceProfile.id),
        ResourceInstanceId = tostring(parse_json(ResourceDetails).instanceDetails.iamInstanceProfile.id),
        ResourceName= tostring(parse_json(ResourceDetails).instanceDetails.instanceId),
        ResourceId= tostring(parse_json(ResourceDetails).instanceDetails.outpostArn),
        AwsActionType= tostring(parse_json(ServiceDetails).action.actionType)
    | extend
        AWSRemoteNetworkIp= coalesce(ApiCallIP,ApiCallIP2, NetworkConnectionIP, InstanceRemoteIp, KubernetesRemoteIp, InstancePORT_PROBERemoteIp, InstanceNETWORK_CONNECTIONRemoteIp, RdsLoginRemoteIp), 
        AWSAssetNetworkIp=coalesce(InstancePrivateIp, KubernetesPrivateIp, "127.0.0.1"), 
        ResourceName = coalesce(ResourceName, ResourceId, eksClusterName, lambdaFunctionName, AwsActionType),
        ResourceUserId =  coalesce(ResourceUserId, ResourceIamInstanceProfileId, KubernetesUserId, EksClusterId, RdsDbUserId, "Na")
        | where not(isempty(AWSRemoteNetworkIp)) and not(isempty(AWSAssetNetworkIp)) and not(ipv4_is_private(AWSRemoteNetworkIp)) 
    | distinct
        TimeGenerated,
        ActivityType,
        ResourceType,
        ResourceUserId,
        ResourceName,
        AWSRemoteNetworkIp,
        AWSAssetNetworkIp,
        Title,
        Description"""

AWSAlerts = qry_prov.exec_query(query)
AWSAlerts = AWSAlerts.to_dataframe()
display(AWSAlerts)

# Explore the TI indicatores in AWS, GCP and Azure

Below cell executions will show you cloud resource access via suspicious IP addresses, where we can find possible compromised users or cloud resources across AWS, GCP, or Azure clouds Env.

In [ ]:
# AWS CloudTrail logs are a great source of information for monitoring and auditing AWS account activity. 

ThreatintelligenceIOCs_IP = list(ThreatintelligenceIOCs[ThreatintelligenceIOCs['IndicatorType'] == 'NetworkIP']['TIIoc'])
ThreatintelligenceIOCs_Domain = list(ThreatintelligenceIOCs[ThreatintelligenceIOCs['IndicatorType'] == 'Domain']['TIIoc'])
ThreatintelligenceIOCs_FileHash = list(ThreatintelligenceIOCs[ThreatintelligenceIOCs['IndicatorType'] == 'FileHash']['TIIoc'])

AWSCloudTrails_IP_View = f"""AWSCloudTrail
            | where TimeGenerated > ago(30d)
        | where SourceIPAddress in ({ThreatintelligenceIOCs_IP})
        | where ActionType == "Success"
        | project
            arn
            EventName,
            EventSource,
            EventTypeName,
            RecipientAccountId,
            ResponseElements,
            SessionMfaAuthenticated,
            SourceIpAddress,
            TimeGenerated,
            UserAgent,
            UserIdentityArn,
            UserIdentityType,
            UserIdentityUserName"""
AWSCloudTrails_IP_View = qry_prov.exec_query(AWSCloudTrails_IP_View)
AWSCloudTrails_IP_View = AWSCloudTrails_IP_View.to_dataframe()
display(AWSCloudTrails_IP_View)

In [ ]:
# GCP Audit logs are a great source of information for monitoring and auditing GCP account activity.

GCPAuditLogs_IP_View = f"""GCPAuditLogs 
            | where TimeGenerated > ago(30d)
      // Extract and transform relevant GCP Audit Log attributes
      | extend
          GCPUserUPN= tostring(parse_json(AuthenticationInfo).principalEmail),
          GCPUserIp = tostring(parse_json(RequestMetadata).callerIp),
          GCPUserUA= tostring(parse_json(RequestMetadata).callerSuppliedUserAgent),
          Tags= tostring(parse_json(Request).tags),
          RequestJS = parse_json(Request)
      | where GCPuserIp in ({ThreatintelligenceIOCs_IP})
      | where GCPUserUPN !has "gserviceaccount.com"
      | extend Name = tostring(split(GCPUserUPN, "@")[0]), UPNSuffix = tostring(split(GCPUserUPN, "@")[1])
      // Select relevant attributes for further analysis
      | project
          GCPOperationTime=TimeGenerated,
          GCPUserUPN,
          GCPUserIp,
          GCPUserUA,
          Name,
          UPNSuffix"""
GCPAuditLogs_IP_View = qry_prov.exec_query(GCPAuditLogs_IP_View)
GCPAuditLogs_IP_View = GCPAuditLogs_IP_View.to_dataframe()  
display(GCPAuditLogs_IP_View)

In [ ]:
# Azure Signin logs are a great source of information for monitoring and auditing Azure account activity.

AzureSiginLogs_IP_View = f"""SigninLogs
            | where TimeGenerated > ago(30d)
            //| where AppDisplayName == "Azure Portal"
            | where isnotempty(OriginalRequestId)
            | where IPAddress in ({ThreatintelligenceIOCs_IP})
            | project 
                IPAddress, 
                UserPrincipalName, 
                TimeGenerated,
                UserAgent,
                ConditionalAccessStatus,
                OperationName,
                RiskDetail,
                AuthenticationRequirement,
                ClientAppUsed 
            // Extracting the name and UPN suffix from UserPrincipalName
            | extend
                Name = tostring(split(UserPrincipalName, "@")[0]),
                UPNSuffix = tostring(split(UserPrincipalName, "@")[1])"""
AzureSiginLogs_IP_View = qry_prov.exec_query(AzureSiginLogs_IP_View)
AzureSiginLogs_IP_View = AzureSiginLogs_IP_View.to_dataframe()
display(AzureSiginLogs_IP_View)

# Find Cross Cload Attacks using Security Tables  

In this section, we are identifying Azure/Microsoft detections based on IP addresses flagged by AWS GuardDuty. This allows us to observe attacker activities across both AWS and Azure cloud resources, providing a broader detection scope for cross-cloud attack tactics, techniques, and procedures (TTPs).

### Explanation of the Kusto Query:

1. **Extract AWS GuardDuty IPs:**
    ```python
    AWSGuarddutyIp = list(AWSAlerts[AWSAlerts['AWSRemoteNetworkIp']])
    ```
    - This line extracts the list of IP addresses flagged by AWS GuardDuty from the `AWSAlerts` dataframe.

2. **Query Azure Alerts:**
    ```python
    AzureAlertsQuery = '''AlertEvidence
    | where EntityType == "Ip"
    | where RemoteIP in ({AWSGuarddutyIp})'''
    ```
    - This Kusto query searches the `AlertEvidence` table in Azure Sentinel for entries where the `EntityType` is "Ip" and the `RemoteIP` matches any of the IP addresses flagged by AWS GuardDuty.

3. **Execute and Display Results:**
    ```python
    AzureAlerts = qry_prov.exec_query(AzureAlertsQuery)
    AzureAlerts = AzureAlerts.to_dataframe()
    display(AzureAlerts)
    ```
    - The query is executed using the `qry_prov` object, and the results are converted into a dataframe and displayed. This shows the Azure alerts that correspond to the suspicious IP addresses detected by AWS GuardDuty.


In [ ]:
AWSGuarddutyIp= list(AWSAlerts[AWSAlerts['AWSRemoteNetworkIp']])
AzureAlertsQuery= '''AlertEvidence
| where EntityType == "Ip"
| where RemoteIP in ({AWSGuarddutyIp})'''
AzureAlerts = qry_prov.exec_query(AzureAlertsQuery)
AzureAlerts = AzureAlerts.to_dataframe()
display(AzureAlerts)


In the above cells, we have explored IP entity-based detections using traditional threat intelligence (TI) sources. These detections are crucial for identifying potential security incidents involving compromised users or cloud resources across AWS, GCP, and Azure environments. By leveraging threat intelligence indicators, we can detect and respond to sophisticated attack tactics, techniques, and procedures (TTPs) that span across different cloud platforms.

### Utilizing Custom Threat Intelligence Feeds

While traditional TI sources provide valuable insights, there are many other techniques and custom TI feeds that can be utilized to enhance cross-cloud attack detection. Custom TI feeds can include data from various sources such as:

- **Internal Threat Intelligence:** Organizations can develop their own threat intelligence feeds based on internal security incidents, logs, and patterns observed within their environment.
- **Third-Party Vendors:** Many organizations use third-party vendors for additional threat intelligence. These vendors provide specialized feeds that can be integrated into the detection process.
- **Open Source Intelligence (OSINT):** Publicly available information from the internet can be used to enrich threat intelligence data.

### Integrating Additional Telemetry Sources

In addition to IP-based detections, integrating telemetry from various sources can provide a more comprehensive view of attacker activities. Some of the key telemetry sources include:

- **Identity Providers (e.g., Okta):** Identity providers offer detailed logs of user authentication and access activities. Analyzing these logs can help identify suspicious login attempts, account takeovers, and other identity-related threats.
- **Network Security Devices (e.g., Palo Alto Networks):** Network security devices provide logs of network traffic, firewall events, and intrusion detection/prevention activities. These logs can be used to detect network-level attacks and anomalies.
- **Endpoint Detection and Response (EDR) Solutions:** EDR solutions provide detailed telemetry from endpoints, including process execution, file access, and network connections. This data is crucial for detecting endpoint-level threats and lateral movement within the network.

### Detecting Attackers' TTPs and Patterns

By combining data from multiple telemetry sources and custom TI feeds, organizations can develop a more robust detection framework. This framework can identify attackers' TTPs and patterns across different cloud environments. Key steps in this process include:

1. **Data Collection:** Collect and aggregate data from various telemetry sources and TI feeds.
2. **Normalization:** Normalize the data to ensure consistency and compatibility across different sources.
3. **Correlation:** Correlate events and indicators from different sources to identify potential attack patterns.
4. **Enrichment:** Enrich the data with additional context from TI feeds and other sources.
5. **Detection:** Develop and apply detection rules and machine learning models to identify suspicious activities and potential threats.
6. **Response:** Implement automated response actions and workflows to mitigate identified threats.


Cross-cloud attack detection requires a comprehensive approach that leverages traditional and custom TI feeds, integrates telemetry from various sources, and identifies attackers' TTPs and patterns. By adopting this approach, organizations can enhance their security posture and effectively protect their multi-cloud environments from sophisticated threats.